In [1]:
import pymysql
import openpyxl
import os

# 경로 설정
os.chdir('../')

from config.GlobalParams import * # DB 접속 정보 불러오기

In [2]:
# 학습 데이터 초기화
def all_clear_train_data(db):
    # 기존 학습 데이터 삭제
    sql = '''
            DELETE FROM chatbot_train_data
        '''
    with db.cursor() as cursor:
        cursor.execute(sql)

    # auto increment 초기화
    sql = '''
    ALTER TABLE chatbot_train_data AUTO_INCREMENT=1
    '''
    with db.cursor() as cursor:
        cursor.execute(sql)

In [3]:
# db에 데이터 저장
def insert_data(db, xls_row):
    # Excel 행을 5개를 컬럼과 대응하는 변수에 저장
    intent, ner, query, answer, answer_img_url = xls_row 

    sql = '''
        INSERT chatbot_train_data(intent, ner, query, answer, answer_image) 
        VALUES(
         '%s', '%s', '%s', '%s', '%s'
        )
    ''' % (intent.value, ner.value, query.value, answer.value, answer_img_url.value)

    # 엑셀에서 불러온 cell에 데이터가 없는 경우, null 로 치환
    sql = sql.replace("'None'", "null")

    with db.cursor() as cursor:
        cursor.execute(sql)
        print('{} 저장'.format(query.value)) # 질문(Query) 출력
        db.commit()

In [4]:
# Excel -> MariaDB
os.chdir('./train_tools/') # 기본 경로 설정
train_file = './train_data.xlsx'
db = None
try:
    db = pymysql.connect(
        host=DB_HOST,
        user=DB_USER,
        passwd=DB_PASSWORD,
        db=DB_NAME,
        charset='utf8'
    )

    # 기존 학습 데이터 초기화, db 연결 객체
    all_clear_train_data(db)

    # 학습 엑셀 파일 불러오기
    wb = openpyxl.load_workbook(train_file)
    sheet = wb['Sheet1'] # 첫번째 Sheet
    # 헤더는 불러오지 않음, 2번째 행부터 읽어옴
    for row in sheet.iter_rows(min_row=2): 
        # 데이터 저장, Excel -> DBMS
        insert_data(db, row)

    wb.close()

except Exception as e:
    print(e)

finally:
    if db is not None:
        db.close()



안녕하세요 저장
반가워요 저장
{B_FOOD} 주문할게요 저장
{B_DT} 예약 저장
None 저장
